# 2025 CITS4012 Group XX Assignment

*Make sure you change the file name with your student id.*

# 0. Readme
这是Project 1的模板，我直接拿来用了

仅需注重代码风格及可视化，让marker改起来舒服就行，分数低不了的
* 例如：多抄Lab的技术栈，多引用学界的基石论文，marker自然乐意审自己熟悉的技术。e.g., WordCloud Visualization

*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object *Oriented* Programming style, please put those the bottom of this ipynb file*

## 0.1 File Structure

```
/content/drive/MyDrive/path_to_your_folder
├── CITS4012_YourGroupID.ipynb
├── sample
├── sample
├── sample
├── sample
├── sample
├── sample
├── test.json
├── train.json
└── validation.json
```

## 0.2 Setup

### Step 1 - Check current Python environment

In [ ]:
import sys
print(sys.executable)

### Step 2 - Install dependencies (2*1 min)

> NOTE:
>
> In Google Drive, an ERROR may occur due to incompatibility of the latest versions of `numpy` and `scipy`.
>
> Simply **restart the runtime** to use the newly downgraded versions.
>
> (In other words, run this code snippet twice to make sure all requirements are satisfied.)

In [ ]:
%pip install gensim

### Step 3 - Mount source files

This step assumes you are running in a Google Colab environment.

In [ ]:
# Define your Google Drive folder path
my_folder = "MyDrive/path_to_your_folder"

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Path setups
import os
folder_path = "/content/drive/" + my_folder

test_file = os.path.join(folder_path, "test.json")
train_file = os.path.join(folder_path, "train.json")
valid_file = os.path.join(folder_path, "validation.json")

# 1. Overview

We implemented three substantially different model architectures:

* [The vanilla RNN-based encoder-decoder](#scrollTo=67TQJgOJ_lF1)

* [The Bi-LSTM encoder-decoder with XXX-attention in different positions](#scrollTo=BhSE5ON4_r0C)

* [The vanilla Transformer with self-attention (no RNN)](#scrollTo=E4w_n2P2_xxm)

# 2. Dataset Processing

# 3. Word Embedding Construction
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

# 4. Model Implementation

## 4.1 The vanilla RNN-based encoder-decoder

## 4.2 The Bi-LSTM encoder-decoder with XXX-attention in different positions

## 4.3 The vanilla Transformer with self-attention

# 5. Performance Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

# 6. Interactive Inference Colab Form
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)